In [1]:
import numpy as np
from keras.datasets import cifar10
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = np.swapaxes(X_train,1,3)
X_train = np.swapaxes(X_train,1,2)

X_test = np.swapaxes(X_test,1,3)
X_test = np.swapaxes(X_test,1,2)

In [3]:
# exercise - implement convnet on Cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD

In [4]:
import tensorflow as tf
tf.python.control_flow_ops = tf

In [5]:
img_rows, img_cols = 32, 32
nb_classes = 10

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,3)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols,3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train,nb_classes)
Y_test = np_utils.to_categorical(y_test,nb_classes)

In [6]:
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(50000, 32, 32, 3) (50000, 10) (10000, 32, 32, 3) (10000, 10)


In [7]:
model = Sequential()
model.add(Convolution2D(32, 3, 3,
                    border_mode='valid',
                    input_shape=(img_rows, img_cols,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
          optimizer='sgd',
          metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=32, 
          nb_epoch=10,verbose=1,
          validation_data=(X_test, Y_test))


#Evaluating the model on the test data    
score, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score)
print('Test accuracy:', accuracy)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 49s - loss: 2.2238 - acc: 0.1566 - val_loss: 2.0275 - val_acc: 0.2689
Epoch 2/10
50000/50000 [==============================] - 67s - loss: 2.0306 - acc: 0.2470 - val_loss: 1.8268 - val_acc: 0.3558
Epoch 3/10
50000/50000 [==============================] - 206s - loss: 1.8564 - acc: 0.3140 - val_loss: 1.6302 - val_acc: 0.4156
Epoch 4/10
50000/50000 [==============================] - 231s - loss: 1.7348 - acc: 0.3624 - val_loss: 1.6105 - val_acc: 0.4207
Epoch 5/10
50000/50000 [==============================] - 239s - loss: 1.6570 - acc: 0.3950 - val_loss: 1.4898 - val_acc: 0.4622
Epoch 6/10
50000/50000 [==============================] - 220s - loss: 1.5864 - acc: 0.4274 - val_loss: 1.3907 - val_acc: 0.4972
Epoch 7/10
50000/50000 [==============================] - 210s - loss: 1.5303 - acc: 0.4492 - val_loss: 1.3617 - val_acc: 0.5139
Epoch 8/10
50000/50000 [=========================

In [8]:
#NOW with data augmentation

In [9]:
from keras.preprocessing.image import ImageDataGenerator


model = Sequential()
model.add(Convolution2D(32, 3, 3,
                    border_mode='valid',
                    input_shape=(img_rows, img_cols,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
          optimizer='sgd',
          metrics=['accuracy'])


print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train,
                    batch_size=32),
                    samples_per_epoch=X_train.shape[0],
                    nb_epoch=10,verbose=1,
                    validation_data=(X_test, Y_test))

Using real-time data augmentation.
Epoch 1/10
50000/50000 [==============================] - 235s - loss: 2.2371 - acc: 0.1530 - val_loss: 2.0362 - val_acc: 0.2695
Epoch 2/10
50000/50000 [==============================] - 239s - loss: 2.0612 - acc: 0.2341 - val_loss: 1.9104 - val_acc: 0.3170
Epoch 3/10
50000/50000 [==============================] - 246s - loss: 1.9129 - acc: 0.2951 - val_loss: 1.7060 - val_acc: 0.3728
Epoch 4/10
50000/50000 [==============================] - 240s - loss: 1.7962 - acc: 0.3333 - val_loss: 1.6302 - val_acc: 0.4118
Epoch 5/10
50000/50000 [==============================] - 248s - loss: 1.7163 - acc: 0.3679 - val_loss: 1.5127 - val_acc: 0.4484
Epoch 6/10
50000/50000 [==============================] - 214s - loss: 1.6389 - acc: 0.3996 - val_loss: 1.4148 - val_acc: 0.4918
Epoch 7/10
50000/50000 [==============================] - 172s - loss: 1.5945 - acc: 0.4210 - val_loss: 1.4937 - val_acc: 0.4723
Epoch 8/10
50000/50000 [==============================] - 204s

In [10]:
#Evaluating the model on the test data    
score, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score)
print('Test accuracy:', accuracy)

Test score: 1.29327878952
Test accuracy: 0.5427
